In [ ]:
import requests
from bs4 import BeautifulSoup as BS

import pandas as pd
import numpy as np
import pickle
import re
from tqdm import tqdm

In [ ]:
missing_reasons = pd.read_pickle('../data/missing_reasons.pkl')

In [ ]:
missing_reasons.sort_values()

In [ ]:
ids = []
reasons = []
notes = []

for r_id in tqdm(missing_reasons):
    url = f'https://www.imdb.com/title/{r_id}/parentalguide?ref_=tt_stry_pg'
    soup = BS(requests.get(url).text)
    
    reason = "None"
        
    try:
        note = re.findall('United States:.+\n.*(?:certificate|No. |pca|#)(.+)\)\n', soup.find('td', attrs = {'class' : ''}).text)
    except:
        note = ["None"]
    
    ids.append(r_id)
    reasons.append(reason)
    notes.append(note)
    
    
reason_df = pd.DataFrame(
    {'imdb_ids' : ids,
     'rating_reasons' : reasons,
     'mpaa_cert_#' : notes}

)

In [ ]:
reasons = reason_df.explode('mpaa_cert_#').reset_index(drop = True)

In [ ]:
reasons['mpaa_cert_#'] = [str(x).strip().replace('#', '') for x in reasons['mpaa_cert_#']]

In [ ]:
reasons[['mpaa_cert_#', 'mpaa_cert_note']] = reasons['mpaa_cert_#'].str.split(',', n=1, expand=True)

In [ ]:
reasons.to_pickle('../data/more_rating_reasons.pkl')